In [61]:
# ! pip install pandas
# ! pip install requests
# ! pip install plotly
# ! pip install datetime
# ! pip install os
# ! pip freeze = requirements.txt

In [62]:
import pandas as pd
import requests as r
import plotly.express as px
import plotly.graph_objects as go
from datetime import datetime, timedelta
import os

In [63]:
pwd = os.getcwd()
if 'L2 TVL' in pwd:
    prepend = ''
else:
    prepend = 'L2 TVL/'

In [64]:

api_str = 'https://api.llama.fi/protocol/'

protocols = [
    # name, incentive start date
         ['velodrome','2022-07-13']
        ,['pooltogether','2022-07-14']
        ,['lyra','2022-08-02']
        ,['rubicon','2022-07-15']
        ,['perpetual-protocol','2022-07-14']
        ,['thales','2022-07-14'] #TVL not relevant
        ,['aave-v3','2022-08-04']
        ,['wepiggy','2022-08-03']
        ,['stargate','2022-08-05']
        ]
# print(protocols[0])
prod = []
for prot in protocols:
    ad = pd.json_normalize( r.get(api_str + prot[0]).json()['chainTvls']['Optimism']['tokens'] )
    ad['date'] = pd.to_datetime(ad['date'], unit ='s') #convert to days
    ad = pd.melt(ad,id_vars = ['date'])
    ad = ad.rename(columns={'variable':'token'})
    ad['token'] = ad['token'].str.replace('tokens.','', regex=False)
    ad['protocol'] = prot[0]
    ad['start_date'] = pd.to_datetime(prot[1])
    ad['date'] = ad['date'] - timedelta(days=1) #change to eod vs sod
    prod.append(ad)

df_df = pd.concat(prod)
df_df

,date,token,value,protocol,start_date
0,2022-06-01 00:00:00,SETH,0.02061,velodrome,2022-07-13
1,2022-06-02 00:00:00,SETH,0.02061,velodrome,2022-07-13
2,2022-06-03 00:00:00,SETH,0.02609,velodrome,2022-07-13
3,2022-06-04 00:00:00,SETH,0.03786,velodrome,2022-07-13
4,2022-06-05 00:00:00,SETH,0.03142,velodrome,2022-07-13
...,...,...,...,...,...
275,2022-08-03 00:00:00,WETH,1881.40751,stargate,2022-08-05
276,2022-08-04 00:00:00,WETH,1518.29696,stargate,2022-08-05
277,2022-08-05 00:00:00,WETH,1326.15808,stargate,2022-08-05
278,2022-08-06 00:00:00,WETH,1007.90122,stargate,2022-08-05


In [65]:
# df_df[df_df['protocol']=='perpetual-protocol']


In [66]:
#defillama api feedback - only token symbols come through, makes it hard to map w/o doing it manually
coingecko_token_map = [
    ['LINK','chainlink']
    ,['USDT','tether']
    ,['USDC','usd-coin']
    ,['WETH','weth']
    ,['SUSD','nusd']
    ,['DAI','dai']
    ,['AAVE','aave']
    ,['WBTC','wrapped-bitcoin']
    ,['SNX','havven']
    ,['OP','optimism']
    ,['SETH','seth']
    ,['FXS','frax-share']
    ,['THALES','thales']
    ,['FRAX','frax']
    ,['ALUSD','alchemix-usd']
    ,['PERP','perpetual-protocol']
    ,['LUSD','liquity-usd']
    ,['LYRA','lyra-finance']
    ,['HND','hundred-finance']
    ,['BITANT','bitant']
    ,['UNI','uniswap']
    ,['SLINK','slink']
    ,['VELO','velodrome-finance']
    ,['DOLA','dola-usd']
    ,['CRV','curve-dao-token']
    ,['SBTC','sbtc']
    ,['KROM','kromatika']
    ,['DF','dforce-token']
    ,['STG','stargate-finance']
    ,['AELIN','aelin']
    ,['RAI','rai']
    ,['RETH','rocket-pool-eth']
]

In [67]:
cg_token_list = [i[0] for i in coingecko_token_map]

In [68]:
# DISTINCT TOKENS

token_list = df_df[['token']].drop_duplicates()
missing_token_list = token_list[~token_list['token'].isin(cg_token_list)]
missing_token_list

,token


In [69]:

cg_pds = []
for t in coingecko_token_map:
    cg_api = 'https://api.coingecko.com/api/v3/coins/'\
            + t[1] + '/market_chart?vs_currency=usd&days=max&interval=daily'
    pr = pd.DataFrame( r.get(cg_api).json()['prices'] )
    pr['token'] = t[0]
    pr['cg_slug'] = t[1]
    pr = pr.rename(columns={0:'date',1:'price_usd'})
    pr['date'] = pd.to_datetime(pr['date'], unit ='ms') #convert to days
    cg_pds.append(pr)

cg_df = pd.concat(cg_pds)
# cg_df

In [70]:
data_df = df_df.merge(cg_df, on=['date','token'],how='inner')
data_df.sort_values(by='date',inplace=True)
data_df['net_token_flow'] = data_df.groupby(['token','protocol','start_date'])['value'].apply(lambda x: x- x.shift(1))
data_df['net_dollar_flow'] = data_df['net_token_flow'] * data_df['price_usd']

data_df

,date,token,value,protocol,start_date,price_usd,cg_slug,net_token_flow,net_dollar_flow
3522,2021-11-02,SUSD,9.448852e+06,lyra,2022-08-02,1.003565,nusd,NaN,NaN
4119,2021-11-02,SBTC,NaN,lyra,2022-08-02,61310.580044,sbtc,NaN,NaN
3908,2021-11-02,SETH,1.211650e+00,lyra,2022-08-02,4360.922300,seth,NaN,NaN
4120,2021-11-03,SBTC,NaN,lyra,2022-08-02,63464.973491,sbtc,NaN,NaN
3523,2021-11-03,SUSD,1.208561e+07,lyra,2022-08-02,1.010893,nusd,2.636756e+06,2.665477e+06
...,...,...,...,...,...,...,...,...,...
1405,2022-08-06,SUSD,1.999396e+06,aave-v3,2022-08-04,1.006770,nusd,-1.218438e+03,-1.226687e+03
1404,2022-08-06,SUSD,2.114768e+05,thales,2022-07-14,1.006770,nusd,-2.238872e+04,-2.254029e+04
1403,2022-08-06,SUSD,1.441349e+07,lyra,2022-08-02,1.006770,nusd,3.083025e+05,3.103898e+05
2292,2022-08-06,UNI,7.535000e-02,velodrome,2022-07-13,8.919415,uniswap,0.000000e+00,0.000000e+00


In [71]:
data_df[data_df['protocol']=='perpetual-protocol'].sort_values(by='date')


,date,token,value,protocol,start_date,price_usd,cg_slug,net_token_flow,net_dollar_flow
5592,2021-12-17,USDC,4.800012e+06,perpetual-protocol,2022-07-14,1.000425,usd-coin,NaN,NaN
5594,2021-12-18,USDC,4.784074e+06,perpetual-protocol,2022-07-14,0.999762,usd-coin,-15938.58904,-15934.802465
5596,2021-12-19,USDC,4.885148e+06,perpetual-protocol,2022-07-14,1.000468,usd-coin,101074.52626,101121.862335
5598,2021-12-20,USDC,4.982993e+06,perpetual-protocol,2022-07-14,0.998758,usd-coin,97844.69300,97723.195069
5600,2021-12-21,USDC,4.948937e+06,perpetual-protocol,2022-07-14,1.003180,usd-coin,-34055.55661,-34163.858349
...,...,...,...,...,...,...,...,...,...
1876,2022-08-02,USDC,2.142510e+07,perpetual-protocol,2022-07-14,1.001107,usd-coin,-210914.69071,-211148.226138
1882,2022-08-03,USDC,2.215844e+07,perpetual-protocol,2022-07-14,0.998803,usd-coin,733333.60583,732455.736932
1888,2022-08-04,USDC,2.238048e+07,perpetual-protocol,2022-07-14,1.002047,usd-coin,222046.62330,222501.240327
1894,2022-08-05,USDC,2.256065e+07,perpetual-protocol,2022-07-14,1.001508,usd-coin,180167.81207,180439.532031


In [72]:
netdf_df = data_df[data_df['date']>= data_df['start_date']][['date','protocol','net_dollar_flow']]
netdf_df = netdf_df.groupby(['date','protocol']).sum(['net_dollar_flow'])
netdf_df['cumul_net_dollar_flow'] = netdf_df.groupby(['protocol']).cumsum()
netdf_df.reset_index(inplace=True)

netdf_df

,date,protocol,net_dollar_flow,cumul_net_dollar_flow
0,2022-07-13,velodrome,1.289995e+06,1.289995e+06
1,2022-07-14,perpetual-protocol,8.598914e+04,8.598914e+04
2,2022-07-14,pooltogether,-1.568344e+04,-1.568344e+04
3,2022-07-14,thales,9.676367e+03,9.676367e+03
4,2022-07-14,velodrome,5.067062e+06,6.357057e+06
...,...,...,...,...
129,2022-08-06,rubicon,3.525457e+05,4.734173e+06
130,2022-08-06,stargate,-1.584665e+06,9.845168e+06
131,2022-08-06,thales,-2.254029e+04,4.940000e+04
132,2022-08-06,velodrome,-1.502144e+06,9.108936e+07


In [73]:
fig = px.line(netdf_df, x="date", y="net_dollar_flow", color="protocol", \
             title="Daily Net Dollar Flow since Program Announcement",\
            labels={
                     "date": "Day",
                     "net_dollar_flow": "Net Dollar Flow (N$F)"
                 }
            )
fig.update_layout(
    legend_title="App Name"
)
fig.update_layout(yaxis_tickprefix = '$')
fig.write_image(prepend + "img_outputs/svg/daily_ndf.svg")
fig.write_image(prepend + "img_outputs/png/daily_ndf.png")

# cumul_fig = px.area(netdf_df, x="date", y="cumul_net_dollar_flow", color="protocol", \
#              title="Cumulative Dollar Flow since Program Announcement",\
#                    labels={
#                      "date": "Day",
#                      "cumul_net_dollar_flow": "Cumulative Net Dollar Flow (N$F)"
#                  }
#             ,areamode='group')
# cumul_fig.update_layout(yaxis_tickprefix = '$')
# cumul_fig.show()


cumul_fig = go.Figure()
proto_names = netdf_df['protocol'].drop_duplicates()
print(proto_names)
for p in proto_names:
    cumul_fig.add_trace(go.Scatter(x=netdf_df[netdf_df['protocol'] == p]['date'] \
                                   , y=netdf_df[netdf_df['protocol'] == p]['cumul_net_dollar_flow'] \
                                    ,name = p\
                                  ,fill='tozeroy')) # fill down to xaxis

cumul_fig.update_layout(yaxis_tickprefix = '$')
cumul_fig.update_layout(
    title="Cumulative Dollar Flow since Program Announcement",
    xaxis_title="Day",
    yaxis_title="Cumulative Net Dollar Flow (N$F)",
    legend_title="App Name",
#     color_discrete_map=px.colors.qualitative.G10
)
cumul_fig.write_image(prepend + "img_outputs/svg/cumul_ndf.svg")
cumul_fig.write_image(prepend + "img_outputs/png/cumul_ndf.png")


0               velodrome
1      perpetual-protocol
2            pooltogether
3                  thales
7                 rubicon
95                   lyra
107               wepiggy
108               aave-v3
121              stargate
Name: protocol, dtype: object


In [74]:
# fig.show()
# cumul_fig.show()
print("yay")

yay


In [75]:
! jupyter nbconvert --to python optimism_app_net_flows.ipynb

[NbConvertApp] Converting notebook optimism_app_net_flows.ipynb to python
[NbConvertApp] Writing 5892 bytes to optimism_app_net_flows.py
